# Aim
Analyze the predicted ages embryos at differente temperatures

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import cv2
import glob
import json
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pathlib
import shutil
import glob
from pathlib import Path
import sys
import re
import seaborn as sns
from scipy import stats
import matplotlib.colors as mcolors
import matplotlib.pylab as pl

from sklearn.linear_model import RANSACRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from scipy import interpolate


np.random.seed(1) # for reproducibility 

# 1. Paths

In [ ]:
# path of json files with data of age predictions 
srcpath = "../temperature_zFish/results/"

# Info dataSets 
dataSetsInfo = "../temperature_zFish/dataSets_info_zfish.txt"
out_json = 'fitData_zfish.json'

# Reference temperature
refTemp = 28.5 + 273.15
T2plot= ['23.5', '28.5', '35.5']


# window_size for smooth interpolation
window_size_frac = 0.0  # fraction of the total time interval

# For stats
central_tendency = 'Median'

# For plotting and fitting 
min_time = 1  # in hpf
max_time = 25 # in hpf
points_per_hour = 12
plotAllData = False
min_valueCbar = 0.0
max_valueCbar = 0.05


# For confidence intervals and RANSAC
min_samples_ransac = 4000 
residual_threshold_ransac = 2.0
num_iterationsBS = 1000 
confidence_level_slope =0.9999

# get refrence dev times
delta_time = max_time-min_time
bins_hist2d = 6*delta_time
ref_time = np.linspace(min_time, max_time, num=points_per_hour*delta_time+1)

# Pattern for files
patternExpAge = r'allData_experimental_age'  
patternEstAge = r'allData_estimated_dev_age' 
patternEstDist = r'allData_CosDist_extimated_age' 
patternEmbryoId = r'allData_fileNames' 

# Read the text file dataSetsInfo as a pandas DataFrame with header
dataImages = pd.read_csv(dataSetsInfo, sep=',', header=0)


# Color for plotting
fullTemprange = np.arange(18,37.0,0.5)
colors = pl.cm.viridis(np.linspace(0,1,len(fullTemprange)))


# Required functions

In [ ]:
def fn_json_load(path_json):
    """Load json file."""
    with open(path_json, 'rb') as JsonFile:
        content = json.load(JsonFile)
    return content

def save_json(data, file_path):
    """Save json file."""
    with open(file_path, 'w') as file:
        # Write the list of numbers to the file in JSON format
        json.dump(data, file)
        
def get_temperature_stamp(file_name):
    parts = file_name.split('_age_')
    temp_stamp = parts[1].split('_')[0]
    temp_dec_stamp = parts[1].split('_')[1]
    temperature = temp_stamp + '.' + temp_dec_stamp
    return temperature     

def get_dataset_name_stamp(file_name):
    parts = file_name.split('_age_')
    name_stamp = parts[1].split('.json')[0]
    return name_stamp 

def generate_croped_list(original_list, P):
    new_list = []
    for element in original_list:
        new_element = element[:P]
        new_list.append(new_element)
    return new_list

def sliding_median_interpolation(x, y, window_size):
    interpolated_y = np.zeros_like(y)
    half_window = window_size // 2

    for i in range(len(x)):
        left_idx = max(0, i - half_window)
        right_idx = min(len(x) - 1, i + half_window)
        window_values = y[left_idx:right_idx + 1]
        interpolated_y[i] = np.median(window_values)

    return interpolated_y

def smoth_embryo_trajectories(X, Y, window_size_frac):

    interpolated_y_vals = []
    for x_val, y_val in zip(X, Y):

        # prepare the data 
        x_val = np.array(x_val) 
        y_val = np.array(y_val) 
        window_size = upper_odd_number(len(x_val) * window_size_frac)           

        # Interpolation 
        interpolated_y_vals.append(sliding_median_interpolation(x_val, y_val, window_size))                        
        
    return interpolated_y_vals    

def create_folder(subfolder_path, folder_name):
    # Join the subfolder path and the folder name
    folder_path = os.path.join(subfolder_path, folder_name)

    try:
        # Create the folder
        os.makedirs(folder_path)
        #print(f"Folder '{folder_name}' created successfully in '{subfolder_path}'!")
    except FileExistsError:
        print(f"Folder '{folder_name}' already exists in '{subfolder_path}'!")
        
        
def upper_odd_number(num):
    return round(num) + 1 if round(num) % 2 == 0 else round(num)     


def bootstrap_ci(x, y, ransac, num_iterations, confidence_level):
    np.random.seed(1) # for reproducibility 
    num_samples = len(x)
    coefs = np.zeros(num_iterations)

    for i in range(num_iterations):
        bootstrap_indices = np.random.choice(num_samples, num_samples, replace=True)
        x_bootstrap = x[bootstrap_indices]
        y_bootstrap = y[bootstrap_indices]
        ransac.fit(x_bootstrap.reshape(-1, 1), y_bootstrap)
        coefs[i] = ransac.estimator_.coef_[0]

        
    lower_percentile = (1 - confidence_level) / 2
    upper_percentile = confidence_level + lower_percentile
    
    lower_bound = np.percentile(coefs, lower_percentile * 100)
    upper_bound = np.percentile(coefs, upper_percentile * 100)
    

    return lower_bound, upper_bound

def bootstrap_embryos_ci(initial_time, X, Y, ransac, num_iterations, confidence_level):
    np.random.seed(0) # for reproducibility 
    num_samples = len(X)
    coefs = np.zeros(num_iterations)

    for i in range(num_iterations):
        bootstrap_indices = np.random.choice(num_samples, num_samples, replace=True)
        
        x_bootstrap = [X[i] for i in bootstrap_indices]
        y_bootstrap = [Y[i] for i in bootstrap_indices]       
        x_bootstrap = np.array(x_bootstrap).flatten() - initial_time
        y_bootstrap = np.array(y_bootstrap).flatten() - initial_time

        ransac.fit(x_bootstrap.reshape(-1, 1), y_bootstrap)
        coefs[i] = ransac.estimator_.coef_[0]

        
    lower_percentile = (1 - confidence_level) / 2
    upper_percentile = confidence_level + lower_percentile
    
    lower_bound = np.percentile(coefs, lower_percentile * 100)
    upper_bound = np.percentile(coefs, upper_percentile * 100)
    

    return lower_bound, upper_bound


def calculate_r2_ransac(x, y, ransac):

    ransac.fit(x.reshape(-1, 1), y)

    inlier_mask = ransac.inlier_mask_
    outlier_mask = np.logical_not(inlier_mask)

    y_inliers = y[inlier_mask]
    y_predicted_inliers = ransac.predict(x[inlier_mask].reshape(-1, 1))
    r2_inliers = r2_score(y_inliers, y_predicted_inliers)
        
    total_inliers = np.sum(inlier_mask)
    total_outliers = np.sum(outlier_mask)
    total_samples = total_inliers + total_outliers

    if total_outliers == 0:
        r2_ransac = r2_inliers
    else:
        y_outliers = y[outlier_mask]
        y_predicted_outliers = ransac.predict(x[outlier_mask].reshape(-1, 1))
        r2_outliers = r2_score(y_outliers, y_predicted_outliers)
        
        r2_ransac = ((total_inliers / total_samples) * r2_inliers) + ((total_outliers / total_samples) * r2_outliers)

    return r2_ransac

def calculate_r2_ransac_inlier(x, y, ransac):

    ransac.fit(x.reshape(-1, 1), y)

    inlier_mask = ransac.inlier_mask_
    outlier_mask = np.logical_not(inlier_mask)

    y_inliers = y[inlier_mask]
    y_predicted_inliers = ransac.predict(x[inlier_mask].reshape(-1, 1))
    r2_inliers = r2_score(y_inliers, y_predicted_inliers)
        

    return r2_inliers


# Custom colormap creation
def custom_viridis_with_zero(cmap_name='custom_viridis_with_zero', zero_color='#EBECF0'):
    cmap = plt.get_cmap('viridis')
    cmap_list = [cmap(i) for i in range(cmap.N)]

    # Find the index corresponding to the lowest color value (0.0) in the colormap
    lowest_idx = 0

    # Set the color for values equal to zero
    cmap_list[lowest_idx] = mcolors.to_rgba(zero_color)

    custom_cmap = mcolors.LinearSegmentedColormap.from_list(cmap_name, cmap_list, cmap.N)
    return custom_cmap

#customized color map
cmap_2Dhist = custom_viridis_with_zero()

# Calculations

In [ ]:
# path of json files with data of age predictions 

# Get a list of all files in the folder
files = sorted(os.listdir(srcpath))

# Filter the files that match the  pattern
experimenAge_files = sorted([file for file in files if re.search(patternExpAge, file)])
estimatedAge_files = sorted([file for file in files if re.search(patternEstAge, file)])
estimatedDist_files = sorted([file for file in files if re.search(patternEstDist, file)])
estimatedNames_files = sorted([file for file in files if re.search(patternEmbryoId, file)])

# Create folder to export the images
create_folder(srcpath, "Averaged_Analysis")
path_averaged = os.path.join(srcpath,"Averaged_Analysis")

# Initialize list to store data
temperature_id = []
absTemps = []
representative_embryo = []
slopes = []
slopesUp = []
slopesDown = []
average_profiles = []
min_profiles = []
max_profiles = []
x_profiles = []
model_pred = []
model_predUp = []
model_predDown = []



# Extact data
for count, file in enumerate(estimatedAge_files, start=0):
    temperature = get_temperature_stamp(file)
    fileExp = experimenAge_files[count]
    fileCosDist = estimatedDist_files[count]
    fileNames = estimatedNames_files[count]
    
    # check that both files (experimental and estimated age) are opened
    
    if get_temperature_stamp(fileExp) == temperature:
        print('*** Processing', temperature, 'C')
        allData_estimated_dev_age = np.array(fn_json_load(os.path.join(srcpath,file))) 
        allData_experimental_age = np.array(fn_json_load(os.path.join(srcpath, fileExp))) 
        allData_CosDist = np.array(fn_json_load(os.path.join(srcpath,fileCosDist)))
        allData_EmbryoNames = fn_json_load(os.path.join(srcpath,fileNames))              

        # Store data temperature
        temperature_id.append(temperature)
        absTemps.append(float(temperature) + 273.15)    

        # Retrieve data form the current dataSet
        dataSet = get_dataset_name_stamp(file)
        res = dataImages[dataImages['DataSetName'] == dataSet]
        if not res.empty:
            img_time_interval = res.iloc[0, 2]
            initial_time =  res.iloc[0, 3]
            last_frame = res.iloc[0, 4]
            NumberEmbryos2Analyze = res.iloc[0, 1]
            print(f"    Dataset: {dataSet} -> timeInterval: {img_time_interval} sec, init_time: {initial_time} hpf, last frame: {last_frame},  {NumberEmbryos2Analyze} embryos")
        else:
            print("The dataSet was not found.")

        # smooth interpolation for each embryo
        if window_size_frac > 0:
            allData_estimated_dev_age = smoth_embryo_trajectories(allData_experimental_age, allData_estimated_dev_age, window_size_frac)
            
            
        # Crop data 
        if len(allData_estimated_dev_age[0]) > last_frame:
            allData_estimated_dev_age = generate_croped_list(allData_estimated_dev_age, last_frame)
            allData_experimental_age = generate_croped_list(allData_experimental_age, last_frame) 
            allData_CosDist = generate_croped_list(allData_CosDist, last_frame)   
        
        
        # Get average for estimated ages and store data       
        if central_tendency == 'Mean':
            avg_values = np.mean(allData_estimated_dev_age, axis=0)
            std_values = np.std(allData_estimated_dev_age, axis=0)
            errorUp = avg_values + std_values
            errorDown = avg_values - std_values
        elif central_tendency == 'Median':
            avg_values = np.median(allData_estimated_dev_age, axis=0)
            # Calculate the absolute deviations from the medians
            abs_deviations = np.abs(allData_estimated_dev_age - avg_values)
            # Calculate the MAD for each column
            mad_values = np.median(abs_deviations, axis=0)
            errorUp = avg_values + 2*mad_values
            errorDown = avg_values - 2*mad_values            
            # Calculate the interquartile range (IQR) of each column
            #iqr_values = stats.iqr(allData_estimated_dev_age, axis=0)
            # Calculate the confidence interval for each column using the median and IQR
            #conf_interval = stats.t.interval(0.95, len(allData_estimated_dev_age)-1, loc=avg_values, scale=iqr_values/1.349)
        else:
            raise ValueError("Invalid central tendency value.")
    
               
        # Calculate the mean squared error for each row relative to the median
        errors = np.linalg.norm(allData_estimated_dev_age - avg_values, axis=1)
        closest_row_index = np.argmin(errors)
        print(f"    Closest embryo to {central_tendency} : {allData_EmbryoNames[closest_row_index]}")

        # Fit linear model
    
        # prepare the data 
        x_val = np.array(allData_experimental_age).flatten() 
        y_val = np.array(allData_estimated_dev_age).flatten()
 
        # move data to (0,0) for fitting   
        x_val = x_val - initial_time
        y_val = y_val - initial_time
        
        # Linear fitting
        ransac_t = RANSACRegressor(estimator=LinearRegression(fit_intercept=False), min_samples=min_samples_ransac, residual_threshold=residual_threshold_ransac)
        ransac_t.fit(x_val.reshape(-1, 1), y_val)       
        slope = ransac_t.estimator_.coef_[0] # Retrieve  fitted model parameters       
#        lower_bound, upper_bound = bootstrap_ci(x_val, y_val, ransac_t, num_iterationsBS, confidence_level_slope) # get 99% confidence intervals        
        lower_bound, upper_bound = bootstrap_embryos_ci(initial_time, allData_experimental_age, allData_estimated_dev_age, ransac_t, num_iterationsBS, confidence_level_slope) # get 99% confidence intervals        
        r2 = calculate_r2_ransac_inlier(x_val,y_val,ransac_t)  # Calculate the R2 score
 
        # restore 
        x_val = x_val + initial_time
        y_val = y_val + initial_time
        
        # Store data 
        representative_embryo.append(allData_EmbryoNames[closest_row_index])
        slopes.append(slope)
        slopesUp.append(upper_bound)
        slopesDown.append(lower_bound)
        average_profiles.append(avg_values)
        min_profiles.append(errorDown)
        max_profiles.append(errorUp)
        x_profiles.append(allData_experimental_age[0])

        
        # Get model predictions
        x_model = ref_time - initial_time
        y_model = ransac_t.predict(x_model.reshape(-1, 1)) + initial_time
        y_down = (lower_bound * x_model) + initial_time 
        y_up = (upper_bound * x_model) + initial_time 
        x_model = ref_time 
        
        model_pred.append(y_model)
        model_predUp.append(y_up)
        model_predDown.append(y_down)
        
        print(f"    Linear fit : {slope:.3f} * Experimental_time ")
        print(f"                 Slope: [{lower_bound:.3f}, {upper_bound:.3f}], R-squared: {r2:.2f}")

        
        # Plot the average curve with standard deviation              
        colorIdx = np.where(fullTemprange == float(temperature))[0][0]
        fig, axs = plt.subplots(dpi=300, figsize=(4,4))
        plt.rcParams['svg.fonttype'] = 'none'
        
        # linear model
        plt.plot(x_model, y_model, color=colors[colorIdx], lw = 1, label='Linear fit: slope={:.3f} [{:.3f},{:.3f}], R2={:.3f}'.format(slope, lower_bound, upper_bound,  r2))
        plt.plot(x_model, y_down, color=colors[colorIdx], lw = 1, ls = '--')
        plt.plot(x_model, y_up, color=colors[colorIdx],lw = 1,  ls = '--')
        # Errors
        plt.fill_between(allData_experimental_age[0], errorUp, errorDown, alpha=0.3, facecolor=colors[colorIdx], label=central_tendency+':'+temperature+'C')        

        if plotAllData == True:
           # reference
            plt.plot(ref_time, ref_time, color='black', linewidth=0.5)
            #density
            plt.hist2d(x_val, y_val, bins=bins_hist2d, range=[[min_time, max_time], [min_time, max_time]], cmap=cmap_2Dhist, density=True, vmin=min_valueCbar, vmax=max_valueCbar)       
            # all data
            plt.scatter(x_val, y_val, s=0.1, color='black', label='Data', edgecolors='none') 
            # average
            plt.plot(allData_experimental_age[0], avg_values, color=colors[count], linewidth=1.0, alpha=0.7, label=central_tendency+':'+temperature+'C')
           #closest embryo
            plt.plot(allData_experimental_age[closest_row_index], allData_estimated_dev_age[closest_row_index], label=allData_EmbryoNames[closest_row_index])       

    
        plt.xlim(0, max_time)
        plt.ylim(0, max_time)
        plt.legend(fontsize='xx-small')
        plt.xlabel('Experimental time (hpf)',  fontsize=8)
        plt.ylabel('Developmental age (hpf)', fontsize=8)
        plt.xticks(np.arange(0, max_time, 4), fontsize=6)          
        plt.yticks(np.arange(0, max_time, 4), fontsize=6)
 #       cbar = plt.colorbar()
 #       cbar.ax.tick_params(labelsize=6) 
        plt.gca().spines['top'].set_visible(False)
        plt.gca().spines['right'].set_visible(False)
       


        outname =  os.path.join(path_averaged, central_tendency+'_'+dataSet+'.svg')
        plt.savefig(outname, bbox_inches='tight', dpi=300)      
    #    plt.show()  
        plt.close()

        
        
        

In [ ]:
# plot examples

plt.rcParams['svg.fonttype'] = 'none'
fig, axs = plt.subplots(dpi=300, figsize=(4,4))

indexesT = [temperature_id.index(item) for item in T2plot]

# reference
#plt.plot(ref_time, ref_time, color='black', linewidth=0.5)

for i, idx in enumerate(indexesT):

    x_val = x_profiles[idx]
    avg_vals = average_profiles[idx]
    down_vals = min_profiles[idx]
    up_vals = max_profiles[idx] 
    temperature = temperature_id[idx]
    slope=slopes[idx]
    lower_bound = slopesDown[idx]
    upper_bound = slopesUp[idx]
    y_model = model_pred[idx]
    y_up =   model_predUp[idx]
    y_down = model_predDown[idx]  
    colorIdx = np.where(fullTemprange == float(temperature))[0][0]
      
    # average
    #plt.scatter(x_val, avg_vals, linewidth=1.0, label=central_tendency+':'+temperature+'C', marker='.', s=1, color=colors[idx])
    plt.fill_between(x_val, down_vals, up_vals, alpha=0.2, label=central_tendency+':'+temperature+'C', facecolor=colors[colorIdx])        
    # linear model
    #plt.plot(x_model, y_model, label='Linear fit: slope={:.3f}, R2={:.3f}'.format(slope, r2))
    plt.plot(ref_time, y_model, color=colors[colorIdx], lw = 1, label='-') #, label='Linear fit: slope={:.3f} [{:.3f},{:.3f}]'.format(slope, lower_bound, upper_bound))
    plt.plot(ref_time, y_down, color=colors[colorIdx], lw = 1, ls = '--')
    plt.plot(ref_time, y_up, color=colors[colorIdx],lw = 1,  ls = '--')
        
plt.xlim(0, max_time)
plt.ylim(0, max_time)
plt.legend(fontsize='xx-small')
plt.xlabel('Experimental time (hpf)',  fontsize=8)
plt.ylabel('Developmental age (hpf)', fontsize=8)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
    

outname =  os.path.join(path_averaged, central_tendency+'_representative_temperatures.svg')
plt.savefig(outname, bbox_inches='tight', dpi=300)      
#    plt.show()  
plt.close()    

In [ ]:
# Arrhenius eq

# prepare data
x = 1 / np.array(absTemps)
y = np.log(slopes)
y_up = np.log(slopesUp)
y_down = np.log(slopesDown)
errors = [y-y_down, y_up-y]

# Perform robust linear fitting using RANSAC
min_s = len(x)
ransac = RANSACRegressor(LinearRegression(), min_samples=min_s, residual_threshold=0.5)
ransac.fit(x.reshape(-1, 1), y)

# Retrieve the inlier mask and fitted model parameters
inlier_mask = ransac.inlier_mask_
outlier_mask = np.logical_not(inlier_mask)
slope = ransac.estimator_.coef_[0]
intercept = ransac.estimator_.intercept_

#plot 
fig, ax = plt.subplots(figsize=(10, 6))

# Create scatter plot
plt.errorbar(x, y, yerr=errors, fmt='o', color='blue', label='data', ms=2)
plt.scatter(x[outlier_mask], y[outlier_mask], color='red', label='Outliers')
plt.plot(x, ransac.predict(x.reshape(-1, 1)), color='green', label='Fit: ln(r/r0) = {:.2f}*(1/T) + {:.2f}'.format(slope, intercept))


# Print the robust linear model parameters
print("Robust Linear Model:")
print(f"Slope: {slope}")
print(f"Intercept: {intercept}")



# Set plot title and labels
plt.xlabel('1/Temperature[1/K]')
plt.ylabel('ln(RelativeGrowthRate)')

ax.set_xticks(x)
ax.set_xticklabels(['1/{:.1f}'.format(val-273.15) for val in absTemps])
plt.legend()

# Save the plot as a high-quality image
outname =  os.path.join(srcpath, 'Arrhenius.png')
plt.savefig(outname, dpi=300)

# Displaying the plot
plt.show()

In [ ]:
# Q 10 rule

from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

refTemp = 18.0 + 273.15
refg = slopes[0]

# Define the function to fit
def exponential_func1(x, a):
    return a**(x/10.0)

# Generate example x and y data
x_data = np.array(absTemps) - refTemp
y_data = slopes / refg
y_up = slopesUp / refg
y_down = slopesDown / refg
errors = [y_data-y_down, y_up-y_data]

# Perform curve fitting
params, params_covariance = curve_fit(exponential_func1, x_data, y_data)

# Extract the fitted parameters
a_fit = params[0]


# Generate the fitted curve
x_fit = np.linspace(np.min(x_data), np.max(x_data), num=100)
y_fit = exponential_func1(x_fit, a_fit)

print(a_fit)
b_fit = 10
fig, ax = plt.subplots(figsize=(10, 6))
# Plot the original data and the fitted curve
plt.errorbar(x_data, y_data, yerr=errors, fmt='o', color='blue', label='Data')
plt.plot(x_fit, y_fit, label=f'Fitting : (r/r0) = {a_fit:.2f}^(dT/{b_fit:.2f})', color='orange')

# Set plot title and labels
plt.xlabel('dT[K]')
plt.ylabel('r/r0')
plt.legend()


# Save the plot as a high-quality image
outname =  os.path.join(srcpath, 'Q10_2.png')
plt.savefig(outname, dpi=300)

# Displaying the plot
plt.show()

In [ ]:
# Create a dictionary from the arrays
data_dict = {
    'slopes': slopes,
    'slopesUp': slopesUp,
    'slopesDown': slopesDown,
    'absTemps': absTemps,
    'temperature_id': temperature_id,
    'representative_embryo': representative_embryo
}

# Create a DataFrame from the dictionary
df = pd.DataFrame(data_dict)

print(df)

# Save the DataFrame as a JSON file
outname =  os.path.join(path_averaged, out_json)
df.to_json(outname, orient='records', lines=True)

In [ ]:
# Plot represnetative embryo at 28.5




# Extact data
for count, file in enumerate(estimatedAge_files, start=0):
    temperature = get_temperature_stamp(file)
    fileExp = experimenAge_files[count]
    fileCosDist = estimatedDist_files[count]
    fileNames = estimatedNames_files[count]
    
    # check that both files (experimental and estimated age) are opened
    
    if get_temperature_stamp(fileExp) == temperature and float(temperature) == 28.5:
        print('*** Processing', temperature, 'C')
        allData_estimated_dev_age = np.array(fn_json_load(os.path.join(srcpath,file))) 
        allData_experimental_age = np.array(fn_json_load(os.path.join(srcpath, fileExp))) 
        allData_CosDist = np.array(fn_json_load(os.path.join(srcpath,fileCosDist)))
        allData_EmbryoNames = fn_json_load(os.path.join(srcpath,fileNames))              

        # Store data temperature
        temperature_id.append(temperature)
        absTemps.append(float(temperature) + 273.15)    

        # Retrieve data form the current dataSet
        dataSet = get_dataset_name_stamp(file)
        res = dataImages[dataImages['DataSetName'] == dataSet]
        if not res.empty:
            img_time_interval = res.iloc[0, 2]
            initial_time =  res.iloc[0, 3]
            last_frame = res.iloc[0, 4]
            NumberEmbryos2Analyze = res.iloc[0, 1]
            print(f"    Dataset: {dataSet} -> timeInterval: {img_time_interval} sec, init_time: {initial_time} hpf, last frame: {last_frame},  {NumberEmbryos2Analyze} embryos")
        else:
            print("The dataSet was not found.")

        # smooth interpolation for each embryo
        if window_size_frac > 0:
            allData_estimated_dev_age = smoth_embryo_trajectories(allData_experimental_age, allData_estimated_dev_age, window_size_frac)
            
            
        # Crop data 
        if len(allData_estimated_dev_age[0]) > last_frame:
            allData_estimated_dev_age = generate_croped_list(allData_estimated_dev_age, last_frame)
            allData_experimental_age = generate_croped_list(allData_experimental_age, last_frame) 
            allData_CosDist = generate_croped_list(allData_CosDist, last_frame)   
        
        
        # Get average for estimated ages and store data       
        if central_tendency == 'Mean':
            avg_values = np.mean(allData_estimated_dev_age, axis=0)
            std_values = np.std(allData_estimated_dev_age, axis=0)
            errorUp = avg_values + std_values
            errorDown = avg_values - std_values
        elif central_tendency == 'Median':
            avg_values = np.median(allData_estimated_dev_age, axis=0)
            # Calculate the absolute deviations from the medians
            abs_deviations = np.abs(allData_estimated_dev_age - avg_values)
            # Calculate the MAD for each column
            mad_values = np.median(abs_deviations, axis=0)
            errorUp = avg_values + 2*mad_values
            errorDown = avg_values - 2*mad_values            
        else:
            raise ValueError("Invalid central tendency value.")
    
               
        # Calculate the mean squared error for each row relative to the median
        errors = np.linalg.norm(allData_estimated_dev_age - avg_values, axis=1)
        closest_row_index = np.argmin(errors)
        print(f"    Closest embryo to {central_tendency} : {allData_EmbryoNames[closest_row_index]}")

        # Get data Embryo 
        x_val = (allData_experimental_age[closest_row_index] - initial_time) * 60
        y_val = (allData_estimated_dev_age[closest_row_index] -initial_time) * 60
        
        # Fit linear model
    
     
        fig, axs = plt.subplots(dpi=300, figsize=(4,4))
        plt.rcParams['svg.fonttype'] = 'none'
        
        # linear model
        plt.scatter(x_val, y_val, color='darkblue', s=1, label=allData_EmbryoNames[closest_row_index])
        plt.plot(np.arange(0, 1440, 200), np.arange(0, 1440, 200), color='red', linewidth=1.0)
            
       
        plt.xlim(0, 1440)
        plt.ylim(0, 1440)
        plt.legend(fontsize='xx-small')
        plt.xlabel('Query embryo time (min)',  fontsize=8)
        plt.ylabel('Reference embryo time (min)', fontsize=8)
        plt.xticks(np.arange(0, 1440, 200), fontsize=6)          
        plt.yticks(np.arange(0, 1440, 200), fontsize=6)
        plt.gca().spines['top'].set_visible(False)
        plt.gca().spines['right'].set_visible(False)
       


        outname =  os.path.join(path_averaged, central_tendency+'_'+dataSet+'_reprensentative_embryo.svg')
        plt.savefig(outname, bbox_inches='tight', dpi=300)      
        plt.close()

        
        